In [1]:
import numpy as np
import xarray as xr
from pathlib import Path
import act
import matplotlib.pyplot as plt
import matplotlib
import glob
%matplotlib inline

print('Reese')

Reese


# Opening a single pblht .cdf file from sgp

Checking available data varibles. We are going to want to filter by the boundary layer regime type (0-neutral)

Basetime is an available data variable, thus we should be able to drop everything that isnt basetime, pbl_regime_type, and pbl_height.

After concatenating a month of .cdf into a single dataset, we should be able to filter (regime=neutral) and store the filtered dataset into a new .cdf file.

In [2]:
ds1 = xr.open_dataset(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\sgppblhtsonde1mcfarlC1.c1.20190430.233200.cdf")
ds1
ds2 = xr.open_dataset(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\sgppblhtsonde1mcfarlC1.c1.20190430.233200.cdf")

In [14]:
ds1

<xarray.Dataset>
Dimensions:                             (height_ss: 189, layer: 5, time: 4615)
Coordinates:
  * time                                (time) datetime64[ns] 2019-05-31T23:2...
  * height_ss                           (height_ss) float32 314.8 ... 2.217e+04
  * layer                               (layer) int32 1 2 3 4 5
Data variables: (12/33)
    base_time                           datetime64[ns] ...
    time_offset                         (time) datetime64[ns] ...
    atm_pres                            (time) float32 ...
    air_temp                            (time) float32 ...
    wspd                                (time) float32 ...
    rh                                  (time) float32 ...
    ...                                  ...
    delta_theta_max                     (layer) float32 ...
    level_1_liu_liang                   float32 ...
    level_2_liu_liang                   float32 ...
    lat                                 float32 ...
    lon                                 float32 ...
    alt                                 float32 ...
Attributes: (12/16)
    command_line:                   idl -D 0 -R -n pblhtsonde -s sgp -f C1 -d...
    process_version:                vap-pblhtsonde-0.8-0.el6
    dod_version:                    pblhtsonde1mcfarl-c1-2.0
    site_id:                        sgp
    platform_id:                    pblhtsonde1mcfarl
    facility_id:                    C1: Lamont, Oklahoma
    ...                             ...
    overshoot_threshold:            4
    reference_1:                    Heffter, J. L., 1980: Transport layer dep...
    reference_2:                    Liu, S. and X. Z. Liang, 2010: Observed D...
    input_datastreams:              sgpsondewnpnC1.b1 : 10.12 : 20190531.0232...
    input_datastreams_description:  A string consisting of the datastream(s),...
    history:                        created by user dsmgr on machine ruby at ...

In [20]:
print(ds1['time'][0].values)

2019-05-31T23:27:00.000000000


In [4]:
#dsa = xr.concat([xr.open_dataset(f) for f in glob.glob(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\*.cdf")], dim='time')
filename = str(Path(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\sgppblhtsonde1mcfarlC1.c1.*.cdf"))
filename


'C:\\Users\\Reese\\Desktop\\AOS Proj Data\\pblht\\sgppblhtsonde1mcfarlC1.c1.*.cdf'

In [5]:
met_ds = act.io.armfiles.read_netcdf(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\sgppblhtsonde1mcfarlC1.c1.*.cdf")
met_ds

TypeError: open_dataset() got an unexpected keyword argument 'combine_attrs'

In [6]:
ds = xr.open_mfdataset(r"C:\Users\Reese\Desktop\AOS Proj Data\pblht\sgppblhtsonde1mcfarlC1.c1.*.cdf", concat_dim='time')

ValueError: Coordinate variable height_ss is neither monotonically increasing nor monotonically decreasing on all datasets

# Possible Option -- 

Might be able to open a new .cdf, loop through the directory containing month's worth of sonde data, check regime type in each iteration and if neutral, pull the pbl height and sonde time to write to new .cdf

In [10]:
import netCDF4 as nc

fn = r"C:\Users\Reese\Desktop\AOS Proj Data\pblhtWrite\test.nc"
ds = nc.Dataset(fn, 'w', format='NETCDF4')
#f4 is 32 bit float i believe 
time = ds.createDimension('time', None)
times = ds.createVariable('time', 'f4', ('time',))
pbl_hgt = ds.createVariable('time', 'f4', ('time',))
pbl_hgt.units = 'meters'

In [21]:
import os

directory = r"C:\Users\Reese\Desktop\AOS Proj Data\pblht"

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        ds1 = xr.open_dataset(f)
        if ds1['pbl_regime_type_liu_liang'].values == 0.0:
            print(ds1['time'][0].values, ds1['pbl_regime_type_liu_liang'].values, ds1['pbl_height_liu_liang'].values)
            

2019-04-30T23:32:00.000000000 0.0 503.8999938964844
2019-05-01T02:29:00.000000000 0.0 515.2999877929688
2019-05-01T05:32:00.000000000 0.0 481.20001220703125
2019-05-01T08:31:00.000000000 0.0 562.7999877929688
2019-05-01T11:31:00.000000000 0.0 539.0999755859375
2019-05-01T14:30:00.000000000 0.0 591.0
2019-05-01T17:28:00.000000000 0.0 776.5
2019-05-01T20:29:00.000000000 0.0 720.2000122070312
2019-05-01T23:30:00.000000000 0.0 584.2999877929688
2019-05-02T02:32:00.000000000 0.0 500.20001220703125
2019-05-02T05:36:00.000000000 0.0 516.5999755859375
2019-05-02T11:30:00.000000000 0.0 490.5
2019-05-02T14:29:00.000000000 0.0 779.2000122070312
2019-05-02T17:31:00.000000000 0.0 905.0999755859375
2019-05-02T20:31:00.000000000 0.0 1081.5
2019-05-02T23:34:00.000000000 0.0 1308.199951171875
2019-05-03T08:41:00.000000000 0.0 563.7000122070312
2019-05-03T11:32:00.000000000 0.0 516.0
2019-05-03T14:31:00.000000000 0.0 649.7000122070312
2019-05-03T20:33:00.000000000 0.0 785.0
2019-05-03T23:38:00.000000000